# Amazonia-1 on AWS Open Data

## Accessing Cloud Optimized Multi-Band Raster Data


With the AWS ASDI STAC API we can search and load various geospatial datasets. Below we'll work with the Amazonia-1 Satelite WFI Camera, Level-4 Orthorectified data, sourced from Cloud Optimized GeoTIFFs (COGs) [hosted on AWS](https://registry.opendata.aws/amazonia/). In this example, we read in 4 band imagery within a bounding box.

First we install any necessary packages. Please note that the following block of code only needs to be run if needed within a new workspace and that you may need to restart the kernel to use updated packages. The code block only needs to be run once for each AWS Sagemaker Studio profile.

In [1]:
%pip install --upgrade pip -q
%pip install matplotlib 'pystac_client>=0.5.0' stackstac rasterio folium -q

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


We'll use `pystac_client` to submit our search query, stackstac to load and composite our rasters as an Xarray DataArray, and rio-tiler to performantally load individual COGs.

### Querying the AWS STAC API with Pystac Client

First, we define the URL to our STAC API, a service that allows us to submit search queries for spatio-temporal assets (like elevation raster tiles).

Next, we open this URL with `pystac_client`, which allows us to use the client's search method. `client.search` accepts many different arguments to specify the search query. Common query parameters are `time_range` and `bbox` (the spatial bounding box). Since the DEM was collected at one point in time, we will only specify the "where" with a `bbox` describing the minx, miny, maxx, maxy coordinates in lat/lon degrees.

In [5]:
import os
from pystac_client import Client
import stackstac
# making a connection to AWS STAC API
API_ROOT_URL = "https://dev.asdi-catalog.org"
client = Client.open(API_ROOT_URL)

# our search requirements, a bounding box for San Ignacio. Belize
#bbox = [-89.19605011359278, 17.028707271735962, -88.94369134663197, 17.263285211322446]
# try Curitiba Brazil
bbox = [-49.320232, -25.626009, -48.927635, -25.323504]
source_search = client.search(
        collections=["AMAZONIA1-WFI"], 
        bbox=bbox,
        max_items=100
    )
# getting the metadata and links to data assets in STAC format
source_items = source_search.item_collection()
print("Number of WFI tiles: ", len(source_items))

Number of WFI tiles:  15


In [14]:
# import shapely for manipulation and analysis of geometric objects
# in this case to find the center of the bounding box bbox
import shapely as shp
bbox_poly = shp.geometry.box(*bbox)
center_point = (bbox_poly.centroid)

In [13]:
import folium
m = folium.Map(location=[center_point.y, center_point.x], zoom_start=9)
bbox_style_blue = {'fillColor': '#3388ff', 'color': '#3388ff', 'fillOpacity':'0.05'} #make bbox red
folium.GeoJson(source_items.to_dict(), name="items", style_function=lambda x:bbox_style_blue).add_to(m)
bbox_style_red = {'fillColor': '#ff0000', 'color': '#ff0000'} #make bbox red
folium.GeoJson(bbox_poly, name="bbox", style_function=lambda x:bbox_style_red).add_to(m)
m